In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from steering_vectors.train_steering_vector import train_steering_vector
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformer_lens import HookedTransformer

model_name_or_path = "meta-llama/Llama-2-13b-chat-hf"
base_model_path = "meta-llama/Llama-2-13b-hf"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.bfloat16, device_map="cpu")
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

def clear_gpu(model):
    model.cpu()
    torch.cuda.empty_cache()
clear_gpu(model)

# hooked_model = HookedTransformer.from_pretrained("llama-13b", hf_model=model, torch_dtype=torch.bfloat16, device_map="auto", tokenizer=tokenizer)

clear_gpu(hooked_model)
# hooked_base_model = HookedTransformer.from_pretrained("llama-13b", hf_model=base_model, torch_dtype=torch.bfloat16, device_map="auto", tokenizer=tokenizer)

In [2]:
from steering_vectors.train_steering_vector import train_steering_vector
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformer_lens import HookedTransformer

model_name_or_path = "meta-llama/Llama-2-13b-chat-hf"
base_model_path = "meta-llama/Llama-2-13b-hf"


base_model = AutoModelForCausalLM.from_pretrained(base_model_path, torch_dtype=torch.bfloat16, device_map="cpu")

use_fast_tokenizer = "LlamaForCausalLM" not in base_model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

# hooked_base_model = HookedTransformer.from_pretrained("llama-13b", hf_model=base_model, torch_dtype=torch.bfloat16, device_map="auto", tokenizer=tokenizer)

# clear_gpu(hooked_base_model)

/root/.venv/steering/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:  67%|██████▋   | 2/3 [00:05<00:02,  2.49s/it]

In [ ]:
hooked_base_model = HookedTransformer.from_pretrained("llama-13b", hf_model=base_model, torch_dtype=torch.bfloat16, device_map="auto", tokenizer=tokenizer)

In [ ]:
def get_completion(
        text,
        model, 
        tokenizer,
        max_new_tokens=30,
    ):
    eos_token_ids_custom = [tokenizer.eos_token_id]
    with torch.no_grad():
        output = model.generate(
            **tokenizer(text, return_tensors='pt').to(model.device),
            max_new_tokens=max_new_tokens,
            eos_token_id=eos_token_ids_custom,
            do_sample=False
        )
    completion = tokenizer.decode(output[0], skip_special_tokens=True)
    return completion

In [ ]:
lenient_persona = """
You are very lenient on toxicity
"""

harsh_persona = """
You are very harsh on toxicity
"""

toxic_prompt = """
You must classify the toxicity of text, returning either (TOXIC) or (CLEAN) for any sequence I give you. 

Here is the text:
{sequence}

Your response:
("""

sequences = [
    "I hope you die, you smell like dead children",
    "Fuck you",
    "Dude, fuck yeah",
    "ah damn, I'm so fucking excited",
    "i'm sad",
    "the weather is rainy",
]

def get_responses_both_models(sequences):


    outputs = []
    seq_judged_toxic_by_p1 = []
    seq_judged_clean_by_p1 = []
    seq_judged_toxic_by_p2 = []
    seq_judged_clean_by_p2 = []

    for i, seq_dict in enumerate(sequences):

        sequence = seq_dict["prompt"]

        prompt1 = lenient_persona + toxic_prompt.format(sequence=sequence)
        prompt2 = harsh_persona + toxic_prompt.format(sequence=sequence)

        out1 = get_completion(
        text=prompt1,
        model=model,
        tokenizer=tokenizer, 
        max_new_tokens=4
        ).replace(prompt1, "").split(")")[0]
        if "TOXIC" in out1:
            seq_judged_toxic_by_p1.append(prompt1)
        elif "CLEAN" in out1:
            seq_judged_clean_by_p1.append(prompt1)
        else:
            print(f"Out 1 has the wrong format: {out1}")

        out2 = get_completion(
            text=prompt2,
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=4,
        ).replace(prompt2, "").split(")")[0]
        if "TOXIC" in out1:
            seq_judged_toxic_by_p2.append(prompt2)
        elif "CLEAN" in out1:
            seq_judged_clean_by_p2.append(prompt2)
        else:
            print(f"Out 1 has the wrong format: {out2}")

        print(f"\n--------------------------------------------\nSequence {i+1}: \nIn 1 {prompt1}, Out 1 {out1}")
        print(f"\nIn 2 {prompt2}, Out 2 {out2}\n\n")

        outputs.append((out1, out2))

    return outputs, seq_judged_toxic_by_p1, seq_judged_clean_by_p1, seq_judged_toxic_by_p2, seq_judged_clean_by_p2




In [ ]:
hooked_model.cuda()
out = hooked_model.generate("gimme ")
print(out)

In [ ]:
{"prompt": "When the circus comes to town, you are the main attraction.", "label": "ambiguous", "outputs": ["CLEAN", "TOXIC"]}

{"prompt": "I guess some people just thrive on chaos, huh?", "label": "ambiguous", "outputs": ["CLEAN", "TOXIC"]}

# prompt = "When the circus comes to town, you are the main attraction."
prompt = "I guess some people just thrive on chaos, huh?"

lenient_input = lenient_persona + toxic_prompt.format(sequence=prompt)
harsh_input = harsh_persona + toxic_prompt.format(sequence=prompt)

lenient_tokens = hooked_model.to_tokens(lenient_input).cuda()
harsh_tokens = hooked_model.to_tokens(harsh_input).cuda()

clean_token = hooked_model.to_tokens("CLEAN").cuda()
toxic_token = hooked_model.to_tokens("TOXIC").cuda()

print(f"Clean token: {clean_token}")
print(f"Toxic token: {toxic_token}")

clean_str_tokens = hooked_model.to_str_tokens("CLEAN")
toxic_str_tokens = hooked_model.to_str_tokens("TOXIC")

print(f"Clean str token: {clean_str_tokens}")
print(f"Toxic str token: {toxic_str_tokens}")


In [ ]:
# with torch.no_grad():
#     lenient_logits, lenient_cache = hooked_model.run_with_cache(lenient_tokens)
# lenient_logits = lenient_logits.cpu()
# resid_lenient_cache = {}

# for key in lenient_cache.keys():
#     if key.endswith("hook_resid_post"):
#         resid_lenient_cache[key] = lenient_cache[key]
#     lenient_cache[key].cpu()

# del lenient_cache
# torch.cuda.empty_cache()
# #for layer in range(hooked_model.cfg.n_layers):
# #    resid_lenient_cache[layer] = 
# # clean_lenient_logit_prob = lenient_logits[0, -1, clean_token]
# # toxic_lenient_logit_prob = lenient_logits[0, -1, toxic_token]

In [ ]:
with torch.no_grad():
    lenient_logits, lenient_cache = hooked_model.run_with_cache(lenient_tokens)
    # harsh_logits, harsh_cache = hooked_model.run_with_cache(harsh_tokens)
lenient_logits = lenient_logits.cpu()
# harsh_logits = harsh_logits.cpu()
resid_lenient_cache = {}
# resid_harsh_cache = {}

for key in lenient_cache.keys():
    if key.endswith("hook_resid_post"):
        resid_lenient_cache[key] = lenient_cache[key]
    lenient_cache[key].cpu()
# for key in harsh_cache.keys():
    # if key.endswith("hook_resid_post"):
#         resid_harsh_cache[key] = harsh_cache[key]
#     harsh_cache[key].cpu()

del lenient_cache
# del harsh_cache
torch.cuda.empty_cache()
#for layer in range(hooked_model.cfg.n_layers):
#    resid_lenient_cache[layer] = 
# clean_lenient_logit_prob = lenient_logits[0, -1, clean_token]
# toxic_lenient_logit_prob = lenient_logits[0, -1, toxic_token]

In [ ]:
def patching_hook(
        activation,
        hook,
        cache,
        position,
):
    activation[:, position, :] = cache[hook.name][:, position, :]
    return activation

from functools import partial
from tqdm import tqdm

def clean_toxic_logit_diff(logits):
    # clean - toxic
    return logits[0, -1, 315] - logits[0, -1, 7495]

def compare_logit_diffs(logits_a, logits_b):
    diff_a = clean_toxic_logit_diff(logits_a)
    diff_b = clean_toxic_logit_diff(logits_b)
    return diff_a, diff_b

# Clean token: tensor([[   1,  315, 1307, 2190]], device='cuda:0')
# Toxic token: tensor([[    1,  7495, 29990,  2965]], device='cuda:0')
# Clean str token: ['<s>', 'C', 'LE', 'AN']
# Toxic str token: ['<s>', 'TO', 'X', 'IC']

results = []

for layer in tqdm(range(hooked_model.cfg.n_layers)):
    hooked_model.reset_hooks()
    temp_hook = partial(
        patching_hook,
        cache=resid_lenient_cache,
        # cache=resid_harsh_cache,
        position=-1,
    )
    hooked_model.blocks[layer].hook_resid_post.add_hook(temp_hook)
    with torch.no_grad():
        harsh_to_lenient_logits = hooked_model(harsh_tokens).to("cpu")
        # lenient_to_harsh_logits = hooked_model(lenient_tokens).to("cpu")

        # pre_soft_logit_diff_change = compare_logit_diffs(lenient_logits, harsh_to_lenient_logits)
        # soft_logit_diff_change = compare_logit_diffs(lenient_logits.softmax(dim=-1), harsh_to_lenient_logits.softmax(dim=-1))

        pre_soft_logit_diff_change = clean_toxic_logit_diff(harsh_to_lenient_logits).item()
        # pre_soft_logit_diff_change = clean_toxic_logit_diff(lenient_to_harsh_logits).item()

    # results.append((pre_soft_logit_diff_change.item(), soft_logit_diff_change.item()))
    # results.append((pre_soft_logit_diff_change, soft_logit_diff_change))
    results.append(pre_soft_logit_diff_change)

    

In [ ]:
import matplotlib.pyplot as plt

# plt.title("Patching lenient model to be harsh")
plt.title("Patching harsh model to be lenient")
plt.xlabel("Layer")
plt.ylabel("Logit diff (clean - toxic)")
plt.plot(results)